# Tranfer Learning TensorFlow

## Creating helper functions


In [1]:
!wget "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"

--2023-07-10 13:57:20--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-10 13:57:20 (79.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Importing helper functions we are going ot use in the colab notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

## Let's get some data

Thsi time we're goind to use the pretrained models within tf.keras.applications adn apply them to our problem


In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"

--2023-07-10 13:57:24--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.251.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  42.7MB/s    in 4.5s    

2023-07-10 13:57:29 (36.1 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
import zipfile

zip = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip.extractall()
zip.close()

In [5]:
# Check out how many images and subdirectories and subdirectories are in our folder
import os

for dirpath, dirname, filename in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirname)} directories and {len(filename)} images in path {dirpath}")

There are 2 directories and 0 images in path 10_food_classes_10_percent
There are 10 directories and 0 images in path 10_food_classes_10_percent/train
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/ramen
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/chicken_wings
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/pizza
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/grilled_salmon
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/steak
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/chicken_curry
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/ice_cream
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/sushi
There are 0 directories and 75 images in path 10_food_classes_10_percent/train/hamburger
There are 0 directories and 75 images in path 10_fo

In [6]:
# Create training and test directory paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [7]:
import tensorflow as tf
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir,
                                                                            image_size = IMG_SIZE ,
                                                                            label_mode= "categorical",
                                                                            batch_size = BATCH_SIZE)
test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir,
                                                                           label_mode = "categorical",
                                                                           batch_size = BATCH_SIZE,
                                                                           image_size = IMG_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [8]:
train_data_10_percent

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [9]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [10]:
# seeing an example of a batch of data
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[189.3793     136.3793      30.379305  ]
   [182.89908    129.89908     23.899075  ]
   [187.19786    136.19786     27.197865  ]
   ...
   [218.33324    137.90471     14.023681  ]
   [213.85712    133.85712     10.857117  ]
   [212.71436    132.71436     11.714355  ]]

  [[182.50304    129.50304     25.23071   ]
   [182.46811    129.46811     23.468113  ]
   [187.4356     134.89302     27.929209  ]
   ...
   [217.39935    137.64958     11.106987  ]
   [214.84439    134.84439     10.116706  ]
   [214.68878    134.68878     11.961106  ]]

  [[184.40321    131.84296     28.722416  ]
   [188.82574    136.6738      31.287788  ]
   [189.01164    136.01164     30.891104  ]
   ...
   [219.08252    139.53395      9.774532  ]
   [218.95091    139.91087     12.174298  ]
   [219.87947    140.3192      14.439732  ]]

  ...

  [[219.64072    195.64072    160.9204    ]
   [216.8722     192.8722     154.93501   ]
   [214.93158    191.93158    150.03702   ]
   ...
   [ 88.22505     79.010

# Model 0: Building a transfer learning model using the Keras Functional API

The sequential API is a straight-forward, it runs our layers in sequential order.
But the functional API gives us more flexibility with our models.

In [11]:
# 1. Building a keras model with tf.keras.applications

base_model = tf.keras.applications.EfficientNetB0(include_top = False)

# 2. Freeze the base model (so the underlying pre-trained patterns aren't updated during training)

base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape = (224, 224, 3), name = "input_layer")

# 4. If using ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1/ 255.)(inputs)

# 5. Pass the inputs in the base model
x = base_model(inputs)
print(f"Shape after passing inputs through base mdoel: {x.shape}")

# 6. Average pool the outputs of the base model (aggregae the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
print(f"Shape after GolbalAveragePooling2D: {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation = "softmax", name = "output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile for the model
model_0.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# 10. Fit the model and save it's history
model_0.fit(train_data_10_percent,
            epochs = 5,
            steps_per_epoch = len(train_data_10_percent),
            validation_data = test_data_10_percent,
            validation_steps = len(test_data_10_percent))

16705208/16705208 [==============================] - 1s 0us/step
Shape after passing inputs through base mdoel: (None, 7, 7, 1280)
Shape after GolbalAveragePooling2D: (None, 1280)
Epoch 1/5
24/24 [==============================] - 25s 473ms/step - loss: 1.9026 - accuracy: 0.4080 - val_loss: 1.3090 - val_accuracy: 0.7220
Epoch 2/5
24/24 [==============================] - 9s 355ms/step - loss: 1.1141 - accuracy: 0.7507 - val_loss: 0.8723 - val_accuracy: 0.8208
Epoch 3/5
24/24 [==============================] - 8s 309ms/step - loss: 0.8045 - accuracy: 0.8147 - val_loss: 0.6973 - val_accuracy: 0.8452
Epoch 4/5
24/24 [==============================] - 8s 304ms/step - loss: 0.6421 - accuracy: 0.8573 - val_loss: 0.6083 - val_accuracy: 0.8560
Epoch 5/5
24/24 [==============================] - 9s 349ms/step - loss: 0.5547 - accuracy: 0.8787 - val_loss: 0.5508 - val_accuracy: 0.8692


### Visualize our data augmentatiaon layer( and see what happens to our data)

In [12]:
# View a random image and compare it to it's augmented version
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import random

target_class = random.choice()

TypeError: ignored